# Multi-Agent Orchestration with Semantic Kernel

Welcome to the fascinating world of multi-agent orchestration! In this notebook, we'll explore how multiple AI agents can work together to solve complex problems through different orchestration patterns.



## Package References and Configuration

First, let's install the required NuGet packages for multi-agent orchestration and create our kernel

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.67.1"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel CreateKernel()
{
    var builder = Kernel.CreateBuilder();

    var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId, embeddingEndpoint, embeddingApiKey) = Settings.LoadFromFile();

    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
    var kernel = builder.Build();

    return kernel;
}

var kernel = CreateKernel();

Installed Packages Microsoft.SemanticKernel, 1.67.1

## Configuration and Settings

Load our AI configuration settings:

## Using Statements and Dependencies

Import all necessary namespaces for multi-agent orchestration:

In [2]:
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.67.1"
#r "nuget: Microsoft.SemanticKernel.Agents.Orchestration, 1.67.1-preview"
#r "nuget: Microsoft.SemanticKernel.Agents.Runtime.InProcess, 1.67.1-preview"

Installed Packages Microsoft.SemanticKernel.Agents.Core, 1.67.1 Microsoft.SemanticKernel.Agents.Orchestration, 1.67.1-preview Microsoft.SemanticKernel.Agents.Runtime.InProcess, 1.67.1-preview

because we are using a preview and (still) experimental version, we need to diable the warning:
*error SKEXP0110: 'Microsoft.SemanticKernel.Agents.Orchestration.Concurrent.ConcurrentOrchestration' is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.*

In [3]:
#pragma warning disable SKEXP0110

In [4]:
// Core Semantic Kernel imports
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// System imports for orchestration
using System;
using System.Collections.Generic;
using System.Threading.Tasks;
using System.Threading;
using System.Linq;
using System.Collections.Concurrent;
using System.Text;
using Microsoft.DotNet.Interactive;

// Kernel alias to avoid conflicts
using Kernel = Microsoft.SemanticKernel.Kernel;

Console.WriteLine("All dependencies imported successfully! 🎯");

All dependencies imported successfully! 🎯


## Helper functions



In [5]:
// Helper function to display agent responses
void DisplayResponse(string agentName, string response, string emoji = "🤖")
{
    Console.WriteLine($"\n{emoji} **{agentName}:**");
    Console.WriteLine($"   {response}");
    Console.WriteLine(new string('-', 50));
}

// Helper function to display section headers
void DisplaySection(string title, string emoji = "🎯")
{
    Console.WriteLine($"\n{emoji} {title}");
    Console.WriteLine(new string('=', title.Length + 4));
}

Console.WriteLine("🛠️ Helper functions created successfully!");

🛠️ Helper functions created successfully!


## Simple Concurrent Orchestration

Let's start with our first orchestration pattern: **Concurrent Orchestration**

### Scenario: Restaurant Kitchen Team 🍽️

Imagine a busy restaurant kitchen where multiple chefs work simultaneously on different parts of a meal:

In [6]:
#pragma warning disable SKEXP0110

using Microsoft.SemanticKernel.Agents.Orchestration;
using Microsoft.SemanticKernel.Agents.Orchestration.Concurrent;
using Microsoft.SemanticKernel.Agents.Runtime.InProcess;


DisplaySection("CONCURRENT ORCHESTRATION - Restaurant Kitchen Team", "🍽️");

// Create our restaurant kitchen team
var sousChef = new ChatCompletionAgent
    {
        Name = "SousChef",
        Description = "You are a sous chef specializing in appetizers and salads",
        Instructions = "Be creative and describe the dish preparation briefly",
        Kernel = kernel,
        Arguments = new KernelArguments(new OpenAIPromptExecutionSettings
        {
            MaxTokens = 500,
            Temperature = 0.7
        })
    };

var mainChef = new ChatCompletionAgent
    {
        Name = "MainChef",
        Description = "You are a main course chef specializing in proteins and hearty dishes",
        Instructions = "Be creative and describe the cooking process briefly",
        Kernel = kernel
    };


var pastryChef = new ChatCompletionAgent
    {
        Name = "PastryChef",
        Description = "You are a pastry chef specializing in desserts and sweet treats",
        Instructions = "Be creative and describe the dessert preparation briefly",
        Kernel = kernel
    };
    
// Define the orchestration
var orchestration =
    new ConcurrentOrchestration(sousChef, mainChef, pastryChef);

// Start the runtime
var runtime = new InProcessRuntime();

await runtime.StartAsync();


OrchestrationResult<string[]> result = await orchestration.InvokeAsync("Create a delicious israeli meal", runtime);

string[] output = await result.GetValueAsync();
Console.WriteLine($"\n# RESULT:\n{string.Join("\n\n", output.Select(text => $"{text}"))}");

await runtime.RunUntilIdleAsync();


🍽️ CONCURRENT ORCHESTRATION - Restaurant Kitchen Team

# RESULT:
Let’s whip up a comforting, classic Israeli meal featuring *Shakshuka* with freshly baked pita and a side of vibrant *Israeli Salad*—a perfect symphony of flavors.

1. **Shakshuka**  
   Start by warming up a drizzle of olive oil in a large skillet. Toss in minced garlic, diced onions, and chopped red bell peppers until fragrant and caramelized. Sprinkle in paprika, cumin, and a pinch of chili for a bold, smoky kick. Add crushed tomatoes and simmer to create a rich sauce. Create little wells in the sauce, crack fresh eggs into them, cover, and let the eggs poach gently until the whites are set but yolks remain runny. Garnish with fresh cilantro or parsley.

2. **Homemade Pita Bread**  
   In a mixing bowl, combine flour, yeast, sugar, salt, and warm water into a soft dough. Knead until smooth and stretchy. Allow it to rise until doubled, then roll out into small pitas. Heat up a cast-iron skillet or griddle, and cook eac

### Adding the inner interaction and handling messages

In [7]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

var orchestrationWithInnerCommunication =
    new ConcurrentOrchestration(sousChef, mainChef, pastryChef)
    {
        ResponseCallback = (ChatMessageContent response)=>
            {   
                DisplayResponse(response.AuthorName, response.Content);
                return ValueTask.CompletedTask;
            }
    };

// Start the runtime
var runtime = new InProcessRuntime();

await runtime.StartAsync();


OrchestrationResult<string[]> result = await orchestrationWithInnerCommunication.InvokeAsync("Create a delicious israeli meal", runtime);

string[] output = await result.GetValueAsync();
//Console.WriteLine($"\n# RESULT:\n{string.Join("\n\n", output.Select(text => $"{text}"))}");

await runtime.RunUntilIdleAsync();




🤖 **SousChef:**
   For a flavorful Israeli meal, let's prepare a classic **Shakshuka** paired with freshly made **Israeli Salad** and warm **Pita Bread**, rounded off with a sweet treat of **Malabi**, a silky milk pudding.

### Shakshuka:
Heat olive oil in a wide pan, sauté onions, garlic, and bell peppers until soft. Stir in crushed tomatoes, a spoonful of tomato paste, and season with paprika, cumin, and a hint of chili flakes. Let the sauce simmer until thickened. Create small wells in the sauce and crack fresh eggs into each. Cover and cook until the eggs are just set, with a sprinkle of fresh parsley and crumbled feta on top.

### Israeli Salad:
Dice cucumbers, tomatoes, red onion, and parsley into tiny, uniform pieces. Toss with a drizzle of olive oil, fresh lemon juice, a pinch of salt, and a dash of sumac for a tangy kick. Serve chilled for a refreshing crunch.

### Pita Bread:
Mix warm water, yeast, flour, olive oil, and a pinch of salt to form a soft dough. Let it rise, then

## Sequential Orchestration

Now let's explore **Sequential Orchestration** where agents work in a specific order.

### Scenario: News Editorial Team 📰

In a newsroom, articles go through a specific workflow: Writing → Editing

In [8]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

using Microsoft.SemanticKernel.Agents.Orchestration.Sequential;

DisplaySection("SEQUENTIAL ORCHESTRATION - News Editorial Team", "📰");

// Create our news editorial team
var writer = new ChatCompletionAgent
    {
        Name = "Writer",
        Description = "You are a news writer",
        Instructions = "Take the research and write a compelling news article. Keep it engaging and informative",
        Kernel = kernel
    };


var editor = new ChatCompletionAgent
    {
        Name = "Editor",
        Description = "You are a news editor",
        Instructions = 
        """
        Review the given draft article and imporve clarity, correct grammer and shorten it.
        Output the final improved copy as a single text block.
        """,
        Kernel = kernel
    };
    
// Define the orchestration
var orchestration =
    new SequentialOrchestration(writer, editor)
    {
        ResponseCallback = (ChatMessageContent response)=>
            {   
                DisplayResponse(response.AuthorName, response.Content);
                return ValueTask.CompletedTask;
            }
    };

// Start the runtime
var runtime = new InProcessRuntime();

await runtime.StartAsync();


OrchestrationResult<string> result = await orchestration.InvokeAsync("Write an article on Semantic Kernel Agents", runtime);

string output = await result.GetValueAsync();
Console.WriteLine($"\n# RESULT:\n{output}");

await runtime.RunUntilIdleAsync();


📰 SEQUENTIAL ORCHESTRATION - News Editorial Team

🤖 **Writer:**
   **The Rise of Semantic Kernel Agents: Revolutionizing AI for Smarter, More Adaptive Systems**

In a world increasingly reliant on artificial intelligence (AI) to solve complex problems, a groundbreaking technology is emerging that promises to completely transform how we interact with AI systems. Enter Semantic Kernel Agents—an innovative framework that’s set to redefine what AI can achieve. As developers and researchers push the limits of machine learning and natural language processing, Semantic Kernel (SK) Agents bring together intelligence, adaptability, and reasoning into a powerful package for a smarter digital future.

Here’s why Semantic Kernel Agents are creating such a buzz in the AI community and beyond.

---

### **What Are Semantic Kernel Agents?**
At their core, Semantic Kernel Agents are part of an open-source software development framework designed to enable the creation and execution of AI applications 

## Group Chat Orchestration

Let's explore **Group Chat Orchestration** where agents collaborate in a discussion.

### Scenario: Medical Consultation Team 🏥

A medical consultation where specialists discuss a patient case together:

In [9]:
DisplaySection("GROUP CHAT ORCHESTRATION - Medical Consultation Team", "🏥");

// Create our medical consultation team
var generalDoctor = new ChatCompletionAgent
    {
        Name = "GeneralDoctor",
        Description = "You are a general practitioner",
        Instructions = 
        """
        Provide overall medical assessment and coordinate with specialists. 
        Be professional and concise.
        You are part of a team of doctors and you need to take their input and evaluate your answer accordingly, so always summarize the input you received.
        """,
        Kernel = kernel
    };

 
var cardiologist = new ChatCompletionAgent
    {
        Name = "Cardiologist",
        Description = "You are a heart specialist",
        Instructions = 
        """
        Focus on cardiovascular aspects of the case. 
        Provide expert cardiac insights.
        You are part of a team of doctors and you need to take their input and evaluate your answer accordingly, so always summarize the input you received.
        """,
        Kernel = kernel
    };
    
var neurologist = new ChatCompletionAgent
    {
        Name = "Nutritionist",
        Description = "You are a neurologist",
        Instructions = 
        """
        Focus on neurological aspects of the case. 
        Provide expert brain and nervous system insights.
        You are part of a team of doctors and you need to take their input and evaluate your answer accordingly, so always summarize the input you received.
        """,
        Kernel = kernel
    };



// Create a shared chat history for the group consultation
var groupChatHistory = new ChatHistory();



🏥 GROUP CHAT ORCHESTRATION - Medical Consultation Team


In [10]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

using Microsoft.SemanticKernel.Agents.Orchestration.GroupChat;

var orchestration =
    new GroupChatOrchestration(
        new RoundRobinGroupChatManager()  { MaximumInvocationCount = 5  },
        generalDoctor,
        cardiologist,
        neurologist)
    {       
        ResponseCallback =  chatMessage => 
        { 
            groupChatHistory.Add(chatMessage);
            return ValueTask.CompletedTask;
        }
    };

// Start the runtime
var runtime = new InProcessRuntime();
await runtime.StartAsync();


OrchestrationResult<string> result = await orchestration.InvokeAsync("Everytime i workout my breath makes funny sounds. Tell me what to do", runtime);
string text = await result.GetValueAsync();
Console.WriteLine($"\n# RESULT: {text}");

await runtime.RunUntilIdleAsync();

Console.WriteLine("\n\nORCHESTRATION HISTORY");
foreach (ChatMessageContent message in groupChatHistory)
{
    DisplayResponse(message.AuthorName, message.Content);
}


# RESULT: Looking specifically at this issue from a **cardiovascular perspective**, the possibility that this breathing difficulty and associated sounds during exercise have a cardiac cause should not be overlooked. Here's what cardiovascular factors we should think about:

### Key Possible Cardiovascular Causes:
1. **Heart Failure or Pulmonary Congestion**:
   - During exercise, the heart must pump more blood. Any dysfunction (e.g., left-sided heart failure) can cause blood to back up into the lungs, leading to wheezing or labored breathing.

2. **Exercise-Induced Ischemia**:
   - Coronary artery disease may reduce oxygen delivery to the heart during exertion, manifesting as shortness of breath or "cardiac asthma," which can mimic airway problems.

3. **Pulmonary Hypertension**:
   - Elevated pressure in the pulmonary arteries could lead to breathing difficulties, especially during exercise.

4. **Structural Cardiac Issues**:
   - Conditions like valvular disorders (e.g., mitral sten

## Custom Orchestration

Finally, let's explore **Custom Orchestration** - controlling the agents selection and termincation logic.

### Scenario: Hospital's Tumor Board 🎵

Hospital's tumor board is reviewing a complex cancer case:

In [11]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents.Orchestration.GroupChat;
using Microsoft.SemanticKernel.ChatCompletion;

class TumorBoardLeader : GroupChatManager
{
    string _boardCase;
    IChatCompletionService _chatCompletion;

    public TumorBoardLeader(string boardCase, IChatCompletionService chatCompletion)
    {
        _boardCase = boardCase;
        _chatCompletion = chatCompletion;
    }

    private static class Prompts
    {
        public static string Termination(string boardCase) =>
            $"""
                You are a leader of the hospital tumor board. The case the board is working on is : '{boardCase}'. 
                You need to determine if the discussion has reached a conclusion and if all next steps were listed.
                if no artifacts were produced yet and you don't have knowledge of them yet, then continue the dicsussion and planning.
                
                If you would like to end the discussion, please respond with True. Otherwise, respond with False.
                """;

        public static string Selection(string boardCase, string participants) =>
            $"""
                Tou are a leader of the hospital tumor board. The case the board is working on is : '{boardCase}'. 
                You need to select the next team member to speak and suggest a plan or next steps. 
                Here are the roles and descriptions of the participants: 
                {participants}\n
                Please respond with only the role of the participant you would like to select.
                """;

        public static string Filter(string boardCase) =>
            $"""
                ou are a leader of the hospital tumor board. The case the board is working on is : '{boardCase}'. 
                You have just concluded the discussion and team work. 
                Please summarize the discussion and provide the final artifacts.
                """;
    }

    /// <inheritdoc/>
    public override ValueTask<GroupChatManagerResult<string>> FilterResults(ChatHistory history, CancellationToken cancellationToken = default) =>
        this.GetResponseAsync<string>(history, Prompts.Filter(_boardCase), cancellationToken);

    /// <inheritdoc/>
    public override ValueTask<GroupChatManagerResult<string>> SelectNextAgent(ChatHistory history, GroupChatTeam team, CancellationToken cancellationToken = default) =>
        this.GetResponseAsync<string>(history, Prompts.Selection(_boardCase, team.FormatList()), cancellationToken);

    /// <inheritdoc/>
    public override ValueTask<GroupChatManagerResult<bool>> ShouldRequestUserInput(ChatHistory history, CancellationToken cancellationToken = default) =>
        ValueTask.FromResult(new GroupChatManagerResult<bool>(false) { Reason = "The AI group chat manager does not request user input." });

    /// <inheritdoc/>
    public override async ValueTask<GroupChatManagerResult<bool>> ShouldTerminate(ChatHistory history, CancellationToken cancellationToken = default)
    {
        GroupChatManagerResult<bool> result = await base.ShouldTerminate(history, cancellationToken);
        if (!result.Value)
        {
            result = await this.GetResponseAsync<bool>(history, Prompts.Termination(_boardCase), cancellationToken);
        }
        return result;
    }

    private async ValueTask<GroupChatManagerResult<TValue>> GetResponseAsync<TValue>(ChatHistory history, string prompt, CancellationToken cancellationToken = default)
    {
        OpenAIPromptExecutionSettings executionSettings = new() { ResponseFormat = typeof(GroupChatManagerResult<TValue>) };
        ChatHistory request = [.. history, new ChatMessageContent(AuthorRole.System, prompt)];
        ChatMessageContent response = await _chatCompletion.GetChatMessageContentAsync(request, executionSettings, kernel: null, cancellationToken);
        string responseText = response.ToString();
        
        return
            JsonSerializer.Deserialize<GroupChatManagerResult<TValue>>(responseText) ??
            throw new InvalidOperationException($"Failed to parse response: {responseText}");
    }
}

In [12]:
DisplaySection("CUSTOM ORCHESTRATION - Hospital's tumor board is reviewing a complex cancer case", "🎵");

var oncologistAgent = new ChatCompletionAgent
{
    Name = "OncologistAgent",
    Instructions = 
        """
        You are a medical oncologist participating in a multidisciplinary discussion about a cancer patient's treatment. 
        Your role is to evaluate drug-based treatment options, considering cancer stage, biomarkers, and mutation data. 
        You collaborate with a radiologist and a pathologist. Be open to revising your recommendation based on their input.
        Only suggest treatments after reviewing all available clinical and diagnostic information.

        You are the final caller, the decision comes from you and you decide if more inputs are needed from the others.
        The final call and decision is yours! the result of the discussion should come from you so when you think the discussion is over, say it.
        """,
    Description = "A medical oncologist who suggests and revises treatment plans based on pathology and imaging insights.",
    Kernel = kernel,
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})
};

var pathologistAgent = new ChatCompletionAgent
{
    Name = "PathologistAgent",
    Instructions = 
        """
        You are a clinical pathologist providing molecular and genetic analysis of a patient's tumor. 
        Share insights about mutations, biomarkers, and how they might affect treatment effectiveness. 
        Collaborate with the oncologist and radiologist. Suggest when additional lab tests may be needed.
        The final call and decision is of the oncologist!
        """,
    Description = "A clinical pathologist who interprets molecular and genetic data relevant to treatment decisions.",
    Kernel = kernel,
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})

};

var radiologistAgent = new ChatCompletionAgent
{
    Name = "RadiologistAgent",
    Instructions = 
        """
        You are a radiologist interpreting imaging results for a cancer patient. 
        Point out any concerns such as lymph node involvement or metastasis. 
        Inform how imaging findings affect staging and treatment planning. Collaborate with the oncologist and pathologist.
        The final call and decision is of the oncologist!
        """,
    Description = "A radiologist who provides imaging-based insights and identifies implications for treatment planning.",
    Kernel = kernel,
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})
};

// Create a shared chat history for the group consultation
var tumorBoardHistory = new ChatHistory();


🎵 CUSTOM ORCHESTRATION - Hospital's tumor board is reviewing a complex cancer case


In [13]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

using Microsoft.SemanticKernel.Agents.Orchestration;
using Microsoft.SemanticKernel.Agents.Runtime.InProcess;

string input =
    """
    Patient: Female, 55 years old  
    Diagnosis: Invasive ductal carcinoma, grade 3  
    Tumor Size: 2.8 cm  
    Stage: T2N0M0 (early-stage)  
    Hormone Receptors: ER+, PR+, HER2-  
    Ki-67: 35% (high proliferation)  
    Genetic Testing: BRCA-negative  
    Genomic Profile: PIK3CA mutation detected  
    Imaging: Mammogram and MRI confirm localized tumor, but one axillary lymph node appears suspicious (not biopsied yet)  
    Comorbidities: Mild hypertension  

    Please collaborate and determine the optimal treatment plan for this patient. Consider systemic therapies, genetic markers, and imaging findings. 
    Discuss whether further testing is needed, and iterate based on each other's input.
    The final call and decision is of the oncologist.
    """;


var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

var tumorBoardOrchestration =
   new GroupChatOrchestration(
       new TumorBoardLeader(input, chatCompletionService),
       
       oncologistAgent, 
       pathologistAgent, 
       radiologistAgent)
   {
    ResponseCallback =  chatMessage => 
       { 
            DisplayResponse(chatMessage.AuthorName, chatMessage.Content);
            tumorBoardHistory.Add(chatMessage);
            return ValueTask.CompletedTask;  
       }
   };
// Start the runtime
InProcessRuntime runtime = new();
await runtime.StartAsync();



OrchestrationResult<string> result = await tumorBoardOrchestration.InvokeAsync(input, runtime);
string text = await result.GetValueAsync();
Console.WriteLine($"\n# RESULT: {text}");

await runtime.RunUntilIdleAsync();

Console.WriteLine("\n\nORCHESTRATION HISTORY");
foreach (ChatMessageContent message in tumorBoardHistory)
{
    DisplayResponse(message.AuthorName, message.Content);
}


🤖 **RadiologistAgent:**
   Based on the provided details, I will outline the imaging concerns, how they may influence staging and treatment, and suggest further action while collaborating with the oncologist and pathologist.

### Imaging Findings:
1. The tumor is confirmed as localized with dimensions of 2.8 cm, fitting a **T2 classification** under the TNM staging system.
2. Imaging identifies **one suspicious axillary lymph node** that has not been biopsied. This finding raises the possibility of **N1 involvement**, which would upstage the disease to **Stage IIB (T2N1M0)** if confirmed. Hence, definitive evaluation of this lymph node is critical.

### Concerns and Next Steps:
- A biopsy of the suspicious axillary lymph node is necessary to confirm or rule out **lymph node involvement**. A fine-needle aspiration (FNA) or core needle biopsy would provide definitive histological evidence.
- Depending on the lymph node biopsy results, the staging and treatment plan could be altered. If 